# Assignment 5 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [220]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [221]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [2]:
# word net installation:

# unmark if you want to use and need to install
!pip install wn
!python -m wn download omw-he:1.4


Cached file found: C:\Users\mim47\.wn_data\downloads\7ecf10e89326bc0ac26ad94b40fe60a7b6ac3302

Checking C:\Users\mim47\AppData\Local\Temp\tmpzi5roocg\omw-he\omw-he.xml
Skipping omw-he:1.4 (Hebrew Wordnet); already added



In [222]:
# word net import:

# unmark if you want to use:
import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [1]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
!pip install hebrew_tokenizer

In [223]:
# Hebrew tokenizer import:

# unmark if you want to use:
import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [119]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [8]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [9]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

# Cleaning and dividing the data

First, we want clean data and the we need to divide the data into X and y.<br>
So i built a cleaning function and i also built a tokinizer for later use:

In [265]:
# auxilery function for cleaning the text from digits and punctuations and prefix
def clean_text(text):
    res = re.sub('[^\w\s]|\d+', '', text)
    res = re.sub('כש+','',text)
    return res



#using the hebrew tokenizer
def tokenizer(text):
    token = ht.tokenize(text)
    tokens = list(filter(lambda x: (x[0] == 'HEBREW'), token))
    tokens = [x[1] for x in tokens]
    return tokens

In [266]:
# cleaning the train set text 
df_train['story'] = df_train['story'].apply(clean_text)


# dividing the df to X_train and y_train
X_train = df_train['story']
y_train = df_train['gender']

# encoding gender to numeric 
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)

In [267]:
print('Training Data :', X_train.shape)

print('Training Data :', y_train.shape)


Training Data : (753,)
Training Data : (753,)


# Text Vectorization

Now we need to vectorize the data.<br>
I'm going to check both the CountVectorizer and TfidfVectorizer:

In [268]:
# Vectorizing the data
Countvec = CountVectorizer(max_df=0.9 ,min_df=5 ,max_features=10000, tokenizer=tokenizer)

X_train_Countvec = Countvec.fit_transform(X_train).toarray()

In [269]:
# Vectorizing the data with TfdifVectorizer
Tfidfvec = TfidfVectorizer(max_df=0.9 ,min_df=5 ,max_features=10000, tokenizer=tokenizer)

X_train_Tfidfvec = Tfidfvec.fit_transform(X_train).toarray()

# Cross Validation

Now i'm goinng to cross validate some models and see what their f1 score is:

In [260]:
#cross val on models
clf_dict = {"Multinomial Naive Bayes":MultinomialNB(), "Gaussian Naive Bayes":GaussianNB(),
            "KNeighbors Classifier":KNeighborsClassifier(), "Decision Tree Classifier":DecisionTreeClassifier(),
            "Perceptron":Perceptron(), "MLP Classifier":MLPClassifier()}


print('using CountVectorizer:')
for key,value in clf_dict.items():
    scores = cross_val_score(value, X_train_Countvec, y_train, scoring='f1_macro', cv=5)
    print("{}: {}".format(key, round(scores.mean(), 3)))
    
print('\n')  
print('-------------------------------') 
print('\n') 

print('using TfidfVectorizer:')
for key,value in clf_dict.items():
    scores = cross_val_score(value, X_train_Tfidfvec, y_train, scoring='f1_macro', cv=5)
    print("{}: {}".format(key, round(scores.mean(), 3)))

using CountVectorizer:
Multinomial Naive Bayes: 0.668
Gaussian Naive Bayes: 0.459
KNeighbors Classifier: 0.539
Decision Tree Classifier: 0.578
Perceptron: 0.667
MLP Classifier: 0.602


-------------------------------


using TfidfVectorizer:
Multinomial Naive Bayes: 0.433
Gaussian Naive Bayes: 0.459
KNeighbors Classifier: 0.597
Decision Tree Classifier: 0.566
Perceptron: 0.579
MLP Classifier: 0.598


So as we can see the combination of Multinomial Naive Bayes model and CountVectorizer got the best f1 score.<br>

# Grid Search

Now i'm going to fine tune the parameters to find the best combination for both Multinomial Naive Bayes model and CountVectorizer:

In [262]:
# creating a pipline and param_grid for the best models and CountVectorizer for grid search

pipe = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', MultinomialNB())
])


param_grid = {
    'vect__ngram_range':[(1, 1), (1, 2)],
    'vect__min_df':[1, 5, 10],
    'vect__max_df':[1.0,0.7, 0.8, 0.9],
    'vect__max_features':[None, 1000, 5000, 10000, 15000],
    'vect__tokenizer':[None, tokenizer],
    'clf__alpha':[0.01, 0.1, 0.5, 1.0, 10.0],
    'clf__fit_prior': [True, False]
}



In [263]:
# performing grid search

gs1 = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1, scoring='f1_macro')
gs1.fit(X_train, y_train)
print(gs1.best_params_)
print('\n')
print(gs1.best_score_)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('clf', MultinomialNB())]),
             n_jobs=-1,
             param_grid={'clf__alpha': [0.01, 0.1, 0.5, 1.0, 10.0],
                         'clf__fit_prior': [True, False],
                         'vect__max_df': [1.0, 0.7, 0.8, 0.9],
                         'vect__max_features': [None, 1000, 5000, 10000, 15000],
                         'vect__min_df': [1, 5, 10],
                         'vect__ngram_range': [(1, 1), (1, 2)],
                         'vect__tokenizer': [None,
                                             <function tokenizer at 0x00000275A3EEB790>]},
             scoring='f1_macro')

{'clf__alpha': 0.5, 'clf__fit_prior': False, 'vect__max_df': 0.8, 'vect__max_features': None, 'vect__min_df': 5, 'vect__ngram_range': (1, 2), 'vect__tokenizer': <function tokenizer at 0x00000275A3EEB790>}


0.6990920406572941


So the best parameters we found are:<br>
for the Multinomial Naive Bayes model: alpha = 0.5, fit_prior = False<br>
for the CountVectorizer: max_df=0.8, max_features=None, min_df=5, ngram_range=(1, 2), tokenizer=tokenizer<br>

Now lets cross validate the pipline with the best parameters found:

In [264]:
# running cross val on the pipline with the best parameters found
pipe = Pipeline([
    ('vect', CountVectorizer(max_df=0.8, max_features=None, min_df=5,
                             ngram_range=(1, 2), tokenizer=tokenizer)),
    ('clf', MultinomialNB(alpha=0.5, fit_prior=False,))])


scores_new = cross_val_score(pipe, X_train, y_train, scoring='f1_macro', cv=5)
print("{}".format(round(scores_new.mean(), 2)))

0.7


I got f1 score of 0.7.

# Prediction

Lets try and predict the gender of the stories in the test set.

In [270]:
# cleaning the test set
X_test = df_test['story'].apply(clean_text)

In [271]:
# predicting the gender of the test set
y_pred = pipe.fit(X_train, y_train).predict(X_test)

# inverse the numeric prediction to 'm' and 'f' using the encoder we encoded the train lable
y_pred = encoder.inverse_transform(y_pred)

df_predicted = pd.DataFrame({'test_example_id':df_test['test_example_id'], 'predicted_category':y_pred})

In [272]:
# first 5 predictions
df_predicted.head(5)

,test_example_id,predicted_category
0,0,m
1,1,m
2,2,m
3,3,m
4,4,m


In [273]:
# last 5 predictions
df_predicted.tail(5)

,test_example_id,predicted_category
318,318,m
319,319,m
320,320,m
321,321,m
322,322,m


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [274]:
df_predicted.to_csv('classification_results.csv',index=False)